
# Comparing randomized search and grid search for hyperparameter estimation

Compare randomized search and grid search for optimizing hyperparameters of a
linear SVM with SGD training.
All parameters that influence the learning are searched simultaneously
(except for the number of estimators, which poses a time / quality tradeoff).

The randomized search and the grid search explore exactly the same space of
parameters. The result in parameter settings is quite similar, while the run
time for randomized search is drastically lower.

The performance is may slightly worse for the randomized search, and is likely
due to a noise effect and would not carry over to a held-out test set.

Note that in practice, one would not search over this many different parameters
simultaneously using grid search, but pick only the ones deemed most important.


### RandomizedSearchCV

#### Overview

`RandomizedSearchCV` is a hyperparameter optimization technique that samples a given number of parameter settings from a specified distribution or set of values, instead of exhaustively searching through all possible combinations as in `GridSearchCV`. This approach allows for a more efficient search, especially when the parameter space is large.

#### How It Works

1. **Define Parameter Distributions**:
   - Instead of defining a grid of parameters, you define a distribution for each parameter. This can be a uniform distribution, a normal distribution, or a list of discrete values.

2. **Random Sampling**:
   - A predefined number of parameter combinations are sampled randomly from these distributions. Each sampled combination is treated as a candidate set of parameters.

3. **Model Evaluation**:
   - Each candidate model is trained and evaluated using cross-validation.

4. **Selection**:
   - The combination of parameters that results in the best performance on the cross-validation set is selected.

#### Mathematical Formulation

Let:
- $P$ be the number of parameters.
- $n_{\text{iter}}$ be the number of iterations (random samples) to be evaluated.
- $\Theta$ represent the parameter space.

The process can be described as follows:

1. **Parameter Sampling**:
   - For each iteration $i$ (where $i \in \{1, \dots, n_{\text{iter}}\}$):
     - Randomly sample a parameter combination $\theta_i \in \Theta$.

2. **Model Training and Evaluation**:
   - For each sampled combination $\theta_i$:
     - Train the model using the parameters $\theta_i$.
     - Evaluate the model's performance using cross-validation and record the score.

3. **Selection**:
   - Select the parameter combination $\theta^*$ that yields the best cross-validation score:
     $$
     \theta^* = \arg\max_{\theta_i \in \{\theta_1, \theta_2, \ldots, \theta_{n_{\text{iter}}}\}} \text{Score}(\theta_i)
     $$

#### Differences from Classic GridSearchCV

- **Efficiency**:
  - `GridSearchCV` evaluates all possible combinations of parameters in the specified grid, which can be computationally expensive.
  - `RandomizedSearchCV` evaluates a fixed number of randomly sampled parameter combinations, which is more efficient for large parameter spaces.

- **Exploration**:
  - `GridSearchCV` can miss optimal hyperparameter values if they lie between the grid points.
  - `RandomizedSearchCV` has a higher chance of finding better hyperparameters since it samples from the entire distribution.

- **Flexibility**:
  - `GridSearchCV` requires the specification of exact grid points for each parameter.
  - `RandomizedSearchCV` allows specifying distributions for parameters, providing more flexibility in the search process.

- **Computational Cost**:
  - `GridSearchCV` can become infeasible when the number of parameter combinations is large.
  - `RandomizedSearchCV` is computationally more efficient, as the number of evaluations is fixed and can be controlled.

### Conclusion

`RandomizedSearchCV` offers a more efficient and flexible approach to hyperparameter optimization compared to `GridSearchCV`, particularly in large parameter spaces. By sampling a fixed number of parameter combinations from specified distributions, it can explore the parameter space more effectively and often converges to good solutions faster than an exhaustive grid search.


In [2]:
from time import time

import numpy as np
import scipy.stats as stats

from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# get some data
X, y = load_digits(return_X_y=True, n_class=3)

# build a classifier
clf = SGDClassifier(loss="hinge", penalty="elasticnet", fit_intercept=True)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {
    "average": [True, False],
    "l1_ratio": stats.uniform(0, 1),
    "alpha": stats.loguniform(1e-2, 1e0),
}

# run randomized search
n_iter_search = 15
random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=n_iter_search
)

start = time()
random_search.fit(X, y)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {
    "average": [True, False],
    "l1_ratio": np.linspace(0, 1, num=10),
    "alpha": np.power(10, np.arange(-2, 1, dtype=float)),
}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print(
    "GridSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(grid_search.cv_results_["params"]))
)
report(grid_search.cv_results_)

RandomizedSearchCV took 0.69 seconds for 15 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.987 (std: 0.018)
Parameters: {'alpha': 0.04826877924427789, 'average': False, 'l1_ratio': 0.8003590258043273}

Model with rank: 2
Mean validation score: 0.987 (std: 0.011)
Parameters: {'alpha': 0.13968474245415072, 'average': False, 'l1_ratio': 0.6120622732189022}

Model with rank: 3
Mean validation score: 0.981 (std: 0.015)
Parameters: {'alpha': 0.036448785708378256, 'average': False, 'l1_ratio': 0.8587670698156694}

GridSearchCV took 2.49 seconds for 60 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.991 (std: 0.012)
Parameters: {'alpha': 0.01, 'average': False, 'l1_ratio': 0.0}

Model with rank: 2
Mean validation score: 0.989 (std: 0.007)
Parameters: {'alpha': 1.0, 'average': False, 'l1_ratio': 0.0}

Model with rank: 3
Mean validation score: 0.989 (std: 0.007)
Parameters: {'alpha': 0.1, 'average': False, 'l1_ratio': 0.0}

Model with rank: 3